# Figure1: Fractional bar plot: met, rxn by compartments, subsystems and lipid sub-subsystems

### Directories

In [1]:
GEM_folder = "..\..\.." # to update if necessary

In [2]:
modelfile = "model\\iMgadit23.json"

### Imports

In [3]:
import cobra
from cobra import Model, Reaction, Metabolite

import json
import pandas as pd
from os.path import join as __join

import matplotlib.pyplot as plt

In [4]:
cobra.show_versions()


System Information
OS         Windows
OS-release      10
Python      3.6.13

Package Versions
appdirs                                      1.4.4
black ; extra == 'development'       not installed
bumpversion ; extra == 'development' not installed
cobra                                       0.21.0
depinfo                                      1.7.0
diskcache                                    5.6.1
future                                      0.18.3
httpx                                       0.22.0
importlib-resources                          5.4.0
isort ; extra == 'development'       not installed
numpy                                       1.19.5
optlang                                      1.4.2
pandas                                       1.1.5
pip                                         21.2.2
pydantic                                     1.9.2
python-libsbml                              5.19.0
rich                                         6.2.0
ruamel.yaml                           

### Model

In [5]:
iMgadit = cobra.io.load_json_model(__join(GEM_folder, modelfile))

In [6]:
iMgadit

Name,iMgadit23
Memory address,0x021c1d43d5f8
Number of metabolites,1977
Number of reactions,2330
Number of groups,0
Objective expression,1.0*BIOMASS_biomass_WT526_c - 1.0*BIOMASS_biomass_WT526_c_reverse_10352
Compartments,"chloroplast stroma, endoplasmic reticulum, extracellular, cytosol, peroxisome, mitochondrial matrix, chloroplast lumen, mitochondrial intermembrane space"


### Functions

In [7]:
def classify_metabolites_by_compartments(model):
    """
    The function classify all metabolites by compartment in a GSM model.
    Results are returned in a dict of list:
        - key: compartment id
        - value: list of metabolite IDs
    """
    dico_met_by_comp = {}

    for comp_id in model.compartments.keys():
        dico_met_by_comp[comp_id]=[]

    for met in model.metabolites:
        met_comp = met.compartment
        dico_met_by_comp[met_comp].append(met.id)
    
    return(dico_met_by_comp)


def count_metabolites_by_compartments(model):
    """
    The function count all metabolites in each compartment in a GSM model.
    Returns a dict:
        - key: compartment id
        - value: number of metabolits in the compartment
    """
    dico_count = {}
    dico_met_by_comp = classify_metabolites_by_compartments(model)
    
    for comp in dico_met_by_comp:
        dico_count[comp]=len(set(dico_met_by_comp[comp]))
    return(dico_count)

In [8]:
def classify_reactions_by_compartments(model):
    """
    The function classify all reactions by compartment in iMgadit GSM model.
    Results are returned in a dict of list:
        - key: compartment id
        - value: list of reactions
    """
    
    # Preparation of dict of results
    dico_rxn_by_comp = {'transport':[],
                       'exchange':[],
                       'biomass':[],
                       'equivalence':[],
                       "others":[]}
    
    for comp_id in model.compartments.keys():
        dico_rxn_by_comp[comp_id]=[]
        
    #Special subsystems
    special_subS = ["Equivalence reaction","Exchange reaction","Transport reaction","Biomass reaction"]

    # Classify model reactions
    for rxn in model.reactions:
        rxn_subS = rxn.subsystem
        if rxn_subS in special_subS:
            dico_rxn_by_comp[rxn_subS.split(" ")[0].lower()].append(rxn.id)

        else:
            rxn_comp = rxn.id[-1]
            if rxn.id[-2] == "_":
                try:
                    dico_rxn_by_comp[rxn_comp].append(rxn.id)
                except:
                    dico_rxn_by_comp["others"].append(rxn.id)
            else:
                dico_rxn_by_comp["others"].append(rxn.id)
        
    return(dico_rxn_by_comp)


def count_reactions_by_compartments(model):
    """
    The function count all reactions in each compartment in iMgadit GSM model.
    Returns a dict:
        - key: compartment id
        - value: number of reactions in the compartment
    """
    dico_count = {}
    dico_rxn_by_comp = classify_reactions_by_compartments(model)
    
    for comp in dico_rxn_by_comp:
        dico_count[comp]=len(set(dico_rxn_by_comp[comp]))
    return(dico_count)


In [9]:

def classify_reactions_by_subsystems(model):
    """
    The function classify all reactions by subsystem in iMgadit GSM model.
    Results are returned in a dict of list:
        - key: compartment id
        - value: list of reactions
    """
    
    # Preparation of dict of results
    dico_rxn_by_subS = {}

    # Classify model reactions
    for rxn in model.reactions:
        subS = rxn.subsystem
        if subS in dico_rxn_by_subS:
            dico_rxn_by_subS[subS].append(rxn.id)
        else:
            dico_rxn_by_subS[subS]=[rxn.id]
    return(dico_rxn_by_subS)

def count_reactions_by_subsystem(model):
    """
    The function count all reactions in each subsystem in iMgadit GSM model.
    Returns a dict:
        - key: compartment id
        - value: reaction count
    """
    dico_count = {}
    dico_rxn_by_subS = classify_reactions_by_subsystems(model)
    
    for comp in dico_rxn_by_subS:
        dico_count[comp]=len(set(dico_rxn_by_subS[comp]))
    return(dico_count)

def classify_reactions_by_lip_kegg(model):
    """
    The function classify all reactions by lipid detailed subsystem in iMgadit GSM model.
    Results are returned in a dict of list:
        - key: sub-subsystem
        - value: list of reactions
    """

    # Preparation of dict of results
    dico_rxn_by_lip_kegg = {}

    # Classify model reactions
    for rxn in model.reactions:
        if rxn.subsystem == 'Lipid metabolism':
            if 'KEGG_Pathway' in rxn.notes:
                lip_kegg = rxn.notes['KEGG_Pathway']
            else:
                lip_kegg = rxn.notes['Other_Pathway']
            if lip_kegg in dico_rxn_by_lip_kegg:
                dico_rxn_by_lip_kegg[lip_kegg].append(rxn.id)
            else:
                dico_rxn_by_lip_kegg[lip_kegg]=[rxn.id]
    return(dico_rxn_by_lip_kegg)

def count_reactions_by_lip_kegg(model):
    """
    The function count all reactions in each lipid sub-subsystem in iMgadit GSM model.
    Returns a dict:
        - key: compartment id
        - value: reaction count
    """
    dico_count = {}
    dico_rxn_by_lip_kegg = classify_reactions_by_lip_kegg(model)
    
    for comp in dico_rxn_by_lip_kegg:
        dico_count[comp]=len(set(dico_rxn_by_lip_kegg[comp]))
    return(dico_count)    

# 1) Reactions and Metabolites by compartment

In [10]:
df_rxn = pd.DataFrame.from_dict(count_reactions_by_compartments(iMgadit),orient='index')

In [11]:
df_rxn

,0
transport,0
exchange,0
biomass,0
equivalence,0
others,519
h,364
r,412
e,0
c,612
p,175


In [12]:
# Remove  transport, export, demand, biomass, eq
not_counted = ["transport", "exchange", "biomass", "equivalence", "others"]

In [13]:
df_rxn_comp = df_rxn.drop(labels =not_counted,axis=0)

In [14]:
df_rxn_comp.columns = ["Count by compartment"]

In [15]:
df_rxn_comp

,Count by compartment
h,364
r,412
e,0
c,612
p,175
m,248
l,0
i,0


In [16]:
cpt_wo_tedbe = 0

for compart in df_rxn_comp.index:
        cpt_wo_tedbe = cpt_wo_tedbe + df_rxn_comp.loc[compart,"Count by compartment"]

In [17]:
# Calculate %
for comp in df_rxn_comp.index:
    cpt = df_rxn_comp.loc[comp,"Count by compartment"]
    percent = 100 * (cpt / cpt_wo_tedbe)
    df_rxn_comp.loc[comp,"Percentage by compartment"] = percent

In [18]:
df_rxn_comp

,Count by compartment,Percentage by compartment
h,364,20.099393
r,412,22.749862
e,0,0.000000
c,612,33.793484
p,175,9.663170
m,248,13.694092
l,0,0.000000
i,0,0.000000


In [19]:
# Test sum % = 100
my_sum = 0 
for comp in df_rxn_comp.index:
    my_sum = my_sum + df_rxn_comp.loc[comp,"Percentage by compartment"]
print(my_sum)

100.0


# Metabolites

In [20]:
df_met = pd.DataFrame.from_dict(count_metabolites_by_compartments(iMgadit),orient='index')

In [21]:
df_met

,0
h,412
r,283
e,47
c,716
p,205
m,306
l,7
i,1


In [22]:
cpt =0
for idx in df_met.index:
    cpt = cpt + df_met.loc[idx,0]

In [23]:
cpt == len(iMgadit.metabolites)

True

In [24]:
df_met.columns = ["Count by compartment"]

In [25]:
cpt_metabo = 0

for compart in df_met.index:
        cpt_metabo = cpt_metabo + df_met.loc[compart,"Count by compartment"]

In [26]:
cpt_metabo

1977

In [27]:
# Calculate %
for comp in df_met.index:
    cpt = df_met.loc[comp,"Count by compartment"]
    percent = 100 * (cpt / cpt_metabo)
    df_met.loc[comp,"Percentage by compartment"] = percent

In [28]:
# Test sum % = 100
my_sum_met = 0 
for comp in df_met.index:
    my_sum_met = my_sum_met + df_met.loc[comp,"Percentage by compartment"]
print(my_sum_met)

100.0


In [29]:
df_met

,Count by compartment,Percentage by compartment
h,412,20.839656
r,283,14.314618
e,47,2.377339
c,716,36.216490
p,205,10.369246
m,306,15.477997
l,7,0.354072
i,1,0.050582


# Format to plot results

In [30]:
dico_comp_id = {'e': 'Extracellular',
 'c': 'Cytosol',
 'h': 'Chloroplast stroma',
 'l': 'Chloroplast lumen',              
 'r': 'Endoplasmic reticulum',
 'p': 'Peroxisome',
 'm': 'Mitochondrial matrix',
 'i': 'Mitochondrial intermembrane space'}

In [31]:
df_res_to_plot = pd.DataFrame()

for compart in dico_comp_id:
    comp_id = dico_comp_id[compart]
    # Reactions
    df_res_to_plot.loc["Reactions",comp_id] = df_rxn_comp.loc[compart,"Count by compartment"]
    # Metabolites
    df_res_to_plot.loc["Metabolites",comp_id] = df_met.loc[compart,"Count by compartment"]

In [32]:
df_res_to_plot

,Extracellular,Cytosol,Chloroplast stroma,Chloroplast lumen,Endoplasmic reticulum,Peroxisome,Mitochondrial matrix,Mitochondrial intermembrane space
Reactions,0.0,612.0,364.0,0.0,412.0,175.0,248.0,0.0
Metabolites,47.0,716.0,412.0,7.0,283.0,205.0,306.0,1.0


In [33]:
df_res_to_plot.to_csv("iMgadit23_Percent_by_compartment.csv",sep=";")

# 2) Reactions by subsystems

In [34]:
classify_reactions_by_subsystems(iMgadit)

{'': ['DGDGS161161_h',
  'EX_arg__L_',
  'ARGDC_c',
  'T_AP_hc_g3p',
  'HISTD2_h',
  'NDPK1_p',
  'NDPK1_m',
  'EX_ppi_',
  'G6PDA_c',
  'DGAT160161204_r',
  'AHC_c',
  'NDPK1_c',
  'DTMPK_m',
  'GLCURS_c',
  'NDPK1_h',
  'DTMPK_c',
  'T_F_hc_13GLUCAN',
  'GLYCL_m',
  'THIPHS_h',
  'T_S_ec_GAL',
  '3HAD040_h',
  'GUAD_c',
  'IMPC_h',
  'IMPC_c',
  'NTD10_c',
  'ACCAH_m',
  'CEPTE204204_r',
  'LACSYN_c',
  'PHYPQOX_h',
  'PGSA161161_h',
  'ARGN_c',
  'PUNP7_c',
  'ETHAK_c',
  'PANTS_m',
  'GLYK_c',
  'DGAT160160161_r',
  '32ECOAI121_p',
  'T_F_rc_H',
  'ACOX181_9Z_p',
  'T_F_rc_DAG205205',
  'ATNS_h',
  'DGTSS140205_r',
  'G3PDq_m',
  'G3PAT204_r',
  'T_F_hc_4ABZ',
  'FTHFL_h',
  'T_F_cm_PYR',
  'GLUDy_c',
  'FTHFL_c',
  'H4THDPS_h',
  'T_AP_mc_ICITSUCC',
  'PAP160161_r',
  'MTRI_c',
  'AMAOT_m',
  'ACS161_c',
  'EX_lys__L_',
  'DAPDC_h',
  'ARD_c',
  'ALDDC_r',
  'T_AP_hc_DHAP',
  'PGCM_h',
  'GLUSfx_h',
  'TRPNR_c',
  'G3PD1_h',
  'PGCM_c',
  'GLUSfx_m',
  'MPTS_c',
  'IAGT_c',
  'EX_

In [35]:
df_subS = pd.DataFrame.from_dict(count_reactions_by_subsystem(iMgadit),orient='index')

In [36]:
df_subS.index

Index([''], dtype='object')

In [37]:
df_subS.to_csv("iMgadit23_rxn_by_subS.csv",sep=";")

# 3) Detailed Kegg for lipid metabolism

In [38]:
classify_reactions_by_lip_kegg(iMgadit)

{}

In [39]:
df_subS_lip = pd.DataFrame.from_dict(count_reactions_by_lip_kegg(iMgadit),orient='index')

In [40]:
df_subS_lip.index

Index([], dtype='object')

In [41]:
df_subS_lip.to_csv("iMgadit23_rxn_detailed_lipid_kegg.csv",sep=";")